In [1]:
import sys,os,os.path
os.environ['NEO4J_BOLT_URL']='bolt://localhost:7687'
os.environ['NEO_PASS']='rl123456'
os.environ['NEO_USER']='neo4j'

In [2]:
import argparse
import os
import torch
import time

from memory_graph.memory_utils import WorkingMemory, ConceptSpaceGDS

from torch import optim
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils import data
from tqdm import tqdm

In [4]:
wm = WorkingMemory(which_db="afftest")
name = wm.create_query_graph('afftest', 'ActionRepr', ['value'])
clusters = wm.compute_action_clusters(f'"{name}"', use_best_k=False, node_property="value")

2.4.3

            CALL gds.beta.kmeans.stream("afftest4ef15d88-3ac2-4918-8bf9-3f52590db6ff", {
            nodeProperty: 'value',
            k: 11,
            randomSeed: 42,
            numberOfRestarts:5
            })
            YIELD nodeId, communityId,distanceFromCentroid
            RETURN gds.util.asNode(nodeId).obj_type AS otype, gds.util.asNode(nodeId).value as emb, communityId, distanceFromCentroid
            ORDER BY communityId, otype ASC, emb, distanceFromCentroid
            


In [5]:
clusters

,otype,emb,communityId,distanceFromCentroid
0,Bucket,"[4.4633206852662255e-11, 1.1563102905731704e-1...",0,1.285542e-10
1,Cannon,"[4.0673898277221454e-11, 9.6867349211327e-12, ...",0,1.237660e-10
2,See_Saw,"[3.7162405097657114e-11, 6.261065450818837e-12...",0,1.310109e-10
3,Trampoline,"[3.30166068096549e-11, 7.752192984766815e-12, ...",0,1.229106e-10
4,Trampoline,"[4.3962517654039246e-11, 1.0978793395233222e-1...",0,2.461925e-10
5,Trampoline,"[4.9084247083497345e-11, 1.4824468042018424e-1...",0,1.483931e-10
6,init,"[0.6507846151082368, 0.12467878570515624, 0.34...",1,0.000000e+00
7,Cannon,"[4.7167967448524806e-11, -2.5858227234790143e-...",2,8.760635e-11
8,Hinge_Constrained,"[4.40232572618271e-11, -2.0247191510197604e-12...",2,8.049452e-11
9,Ramp,"[3.7635332350571815e-11, 2.5595302786951546e-1...",2,1.417729e-10


In [6]:
dfs=[]
maxs=[]
for i in range(11):
    dfs.append(clusters[clusters['communityId']==i]['otype'].value_counts().to_frame())
    maxs.append(clusters[clusters['communityId']==i]['otype'].value_counts().idxmax())

    

In [7]:
maxs

['Trampoline',
 'init',
 'Ramp',
 'Ramp',
 'See_Saw',
 'Ramp',
 'Cannon',
 'Hinge_Constrained',
 'Trampoline',
 'Bouncy_Ball',
 'Bucket']

In [8]:
dfs

[            otype
 Trampoline      3
 Bucket          1
 Cannon          1
 See_Saw         1,
       otype
 init      1,
                    otype
 Ramp                   2
 See_Saw                2
 Cannon                 1
 Hinge_Constrained      1,
          otype
 Ramp         2
 See_Saw      1,
          otype
 See_Saw      2
 Cannon       1,
             otype
 Ramp            2
 Bucket          1
 Cannon          1
 Fixed_Ball      1,
             otype
 Cannon          3
 Trampoline      2
 Bucket          1
 Fixed_Ball      1
 Ramp            1,
                    otype
 Hinge_Constrained      1,
             otype
 Trampoline      3
 Fixed_Ball      1,
              otype
 Bouncy_Ball      1
 Ramp             1,
         otype
 Bucket      1]

In [9]:
cluster_emb= list(clusters['emb'])

In [ ]:
from sklearn.manifold import TSNE

# The default of 1,000 iterations gives fine results, but I'm training for longer just to eke
# out some marginal improvements. NB: This takes almost an hour!
tsne = TSNE(random_state=1, n_iter=15000, metric="cosine")

embs = tsne.fit_transform(cluster_emb)
# Add to dataframe for convenience
x = embs[:, 0]
y = embs[:, 1]

In [ ]:
import matplotlib.pyplot as plt

FS = (10, 8)
fig, ax = plt.subplots(figsize=FS)
# Make points translucent so we can visually identify regions with a high density of overlapping points
ax.scatter(x, y, alpha=.1);

In [ ]:
from sklearn.manifold import TSNE
import seaborn as sns

X_embedded = TSNE(n_components=2,random_state=42).fit_transform(cluster_emb)
centers = np.array(kmeans.cluster_centers_)
model = KMeans(n_clusters = 4, init = "k-means++")
label = model.fit_predict(X_embedded)


plt.figure(figsize=(10,10))
uniq = np.unique(label)
for i in uniq:
   plt.scatter(data[label == i , 0] , data[label == i , 1] , label = i)
plt.scatter(centers[:,0], centers[:,1], marker="x", color='k')
#This is done to find the centroid for each clusters.
plt.legend()
plt.show()